# Semantic Search


## 필요한 패키지 설치


In [224]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

## 데이터 준비


In [245]:
import pandas as pd

movies_raw_df = pd.read_csv("./data/movies.csv", low_memory=False)
movies_raw_df.head(10)

title          genre             year  date   rating  vote_count  plot                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               main_act                                                        supp_act                                                                                                                                                                       page_url                                                   img_url                                                                                            
...ing         멜로/로맨스|드라마        2003  11.28  8.89   

## OpenSearch에 데이터 인덱싱하기


In [419]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region="us-east-1")

aos_host = "search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws"


aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

## 임베딩 모델 준비하기


In [227]:
s = b'{"transient":{"plugins.ml_commons.only_run_on_ml_node": false}}'
aos_client.cluster.put_settings(body=s)

{'acknowledged': True,
 'persistent': {},
 'transient': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false'}}}}

### Bedrock Titan Embedding G1 모델 Register

request 패키지를 이용해본다.


In [228]:
# Bedrock Titan Embeddings G1 - Text Connector 생성

import requests

connector_payload = {
    "name": "Connector to Bedrock Titan Embeddings G1 - Text",
    "description": "The connector to Bedrock Titan Embeddings G1 - Text",
    "version": 1,
    "protocol": "aws_sigv4",
    "parameters": {
        "region": "us-east-1",
        "service_name": "bedrock",
        "input_docs_processed_step_size": 2,
    },
    "credential": {"roleArn": "arn:aws:iam::236241703319:role/OpenSearchForBedrockRole"},
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/amazon.titan-embed-text-v1/invoke",
            "headers": {"content-type": "application/json", "x-amz-content-sha256": "required"},
            "request_body": '{ "inputText": "${parameters.inputText}" }',
            "pre_process_function": '\n    StringBuilder builder = new StringBuilder();\n    builder.append("\\"");\n    String first = params.text_docs[0];\n    builder.append(first);\n    builder.append("\\"");\n    def parameters = "{" +"\\"inputText\\":" + builder + "}";\n    return  "{" +"\\"parameters\\":" + parameters + "}";',
            "post_process_function": '\n      def name = "sentence_embedding";\n      def dataType = "FLOAT32";\n      if (params.embedding == null || params.embedding.length == 0) {\n        return params.message;\n      }\n      def shape = [params.embedding.length];\n      def json = "{" +\n                 "\\"name\\":\\"" + name + "\\"," +\n                 "\\"data_type\\":\\"" + dataType + "\\"," +\n                 "\\"shape\\":" + shape + "," +\n                 "\\"data\\":" + params.embedding +\n                 "}";\n      return json;\n    ',
        }
    ],
}

base_url = (
    "https://search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws/_plugins/_ml"
)

# Send the request
response = requests.post(
    base_url + "/connectors/_create",
    auth=auth,
    json=connector_payload,
)

# Print the response
print(response.text)

{"connector_id":"iD1Qjo4B1FbzESchRpWr"}


In [229]:
obj = json.loads(response.text)
connector_id = obj["connector_id"]

Register Model Group


In [230]:
register_model_group_payload = {
    "name": "Bedrock Models",
    "description": "This is a model group for Amazon Bedrock remote model.",
}

# Send the request
response = requests.post(
    base_url + "/model_groups/_register",
    auth=auth,
    json=register_model_group_payload,
)

# Print the response
print(response.text)

{"model_group_id":"EVZQjo4B5vU2gwgcSrgY","status":"CREATED"}


In [231]:
obj = json.loads(response.text)
model_group_id = obj["model_group_id"]

Register Model


In [232]:
register_model_payload = {
    "name": "Bedrock text embedding model",
    "function_name": "remote",
    "model_group_id": model_group_id,
    "description": "This is Bedrock Titan Embeddings G1 - Text model",
    "connector_id": "flaMhY4B5vU2gwgcXTu4",
}

# Send the request
response = requests.post(
    base_url + "/models/_register",
    auth=auth,
    json=register_model_payload,
)

# Print the response
print(response.text)

{"task_id":"iT1Qjo4B1FbzESchTZWD","status":"CREATED","model_id":"ij1Qjo4B1FbzESchTZWr"}


In [233]:
obj = json.loads(response.text)
model_id = obj["model_id"]
print(model_id)

ij1Qjo4B1FbzESchTZWr


Deploy Model


In [234]:
deploy_model_payload = {}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_deploy",
    auth=auth,
    json=deploy_model_payload,
)

# Print the response
print(response.text)

{"task_id":"ElZQjo4B5vU2gwgcUbgf","task_type":"DEPLOY_MODEL","status":"COMPLETED"}


Test the model


In [235]:
test_model_payload = {"parameters": {"inputText": "바다에서 펼쳐지는 해적들의 꿈과 모험 이야기"}}

# Send the request
response = requests.post(
    base_url + "/models/" + "kGhNiI4BnEOV7ezVxzHF" + "/_predict",
    auth=auth,
    json=test_model_payload,
)

# Print the response
display(json.loads(response.text))

{'inference_results': [{'output': [{'name': 'sentence_embedding',
     'data_type': 'FLOAT32',
     'shape': [1536],
     'data': [0.0075683594,
      -0.80078125,
      0.0054016113,
      0.23828125,
      0.68359375,
      0.2109375,
      -0.12207031,
      -0.0005187988,
      -0.94140625,
      -0.84375,
      0.12011719,
      0.59375,
      -0.17382812,
      1.1953125,
      0.19335938,
      -0.40429688,
      -0.23242188,
      -0.14550781,
      -0.42382812,
      -0.38476562,
      0.41601562,
      -0.5234375,
      -0.15625,
      1.203125,
      -0.20996094,
      0.25976562,
      0.35546875,
      -1.21875,
      -0.029907227,
      0.18554688,
      -0.83203125,
      1.0703125,
      0.37109375,
      0.31054688,
      -0.04663086,
      -0.33398438,
      -0.13769531,
      -0.44335938,
      0.18066406,
      -0.056884766,
      -0.73828125,
      -0.74609375,
      -0.24707031,
      0.47070312,
      -0.056640625,
      0.48046875,
      0.75390625,
      0.0327

In [236]:
print(model_id)

ij1Qjo4B1FbzESchTZWr


## Ingest Pipeline 생성


In [237]:
# aos_client.ingest.get_pipeline(id="embedding_pipeline")
aos_client.indices.delete(index="movies_semantic_index")

{'acknowledged': True}

In [238]:
pipeline = {
    "description": "An neural search pipeline - titan embedding g1 text",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "plot": "plot_vector",
                },
            }
        },
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "title": "title_vector",
                },
            }
        },
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "genre": "genre_vector",
                },
            }
        },
    ],
}

pipeline_id = "embedding_pipeline"
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

In [239]:
aos_client.ingest.get_pipeline(id=pipeline_id)

{'embedding_pipeline': {'description': 'An neural search pipeline - titan embedding g1 text',
  'processors': [{'text_embedding': {'model_id': 'ij1Qjo4B1FbzESchTZWr',
     'field_map': {'plot': 'plot_vector'}}},
   {'text_embedding': {'model_id': 'ij1Qjo4B1FbzESchTZWr',
     'field_map': {'title': 'title_vector'}}},
   {'text_embedding': {'model_id': 'ij1Qjo4B1FbzESchTZWr',
     'field_map': {'genre': 'genre_vector'}}}]}}

### Index 생성


In [240]:
movies_semantic_index = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "cosinesimil",
    },
    "mappings": {
        "properties": {
            "date": {"type": "float"},
            "genre": {
                "type": "text",
            },
            "img_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "page_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
            "title_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
            "plot_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
            "genre_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

In [241]:
index_name = "movies_semantic_index"

# aos_client.indices.delete(index=index_name)
aos_client.indices.create(index=index_name, body=movies_semantic_index)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'movies_semantic_index'}

In [242]:
aos_client.indices.get(index=index_name)

{'movies_semantic_index': {'aliases': {},
  'mappings': {'properties': {'date': {'type': 'float'},
    'genre': {'type': 'text'},
    'genre_vector': {'type': 'knn_vector',
     'store': True,
     'dimension': 1536,
     'method': {'engine': 'faiss',
      'space_type': 'l2',
      'name': 'hnsw',
      'parameters': {}}},
    'img_url': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'main_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'page_url': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'plot': {'type': 'text'},
    'plot_vector': {'type': 'knn_vector',
     'store': True,
     'dimension': 1536,
     'method': {'engine': 'faiss',
      'space_type': 'l2',
      'name': 'hnsw',
      'parameters': {}}},
    'rating': {'type': 'float'},
    'supp_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
 

## Load data


In [243]:
# Try ingest one document

# payload = {
#     "title": "파묘",
#     "genre": "공포",
#     "image_url": "http://pamyo.com",
#     "page_url": "http://pamyo.com",
#     "main_act": "최민식, 류해진",
#     "plot": "아주 무서운 이야기",
#     "rating": "10.0",
#     "supp_act": "안기범, 권혁수",
#     "vote_count": "100",
#     "year": "2024",
# }

# aos_client.index(index=index_name, body=payload)

In [244]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = movies_raw_df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=2, queue_size=2
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)


# Ingest the DataFrame
# for idx, row in tqdm(movies_raw_df.iterrows()):
#     aos_client.index(index="movies_default_index", body=row.to_dict(), id=idx)

BulkIndexError: ('1 document(s) failed to index.', [{'index': {'_index': 'movies_semantic_index', '_id': None, 'status': 400, 'error': {'type': 'illegal_argument_exception', 'reason': 'Invalid JSON in payload'}, 'data': '{"title":"\\uac8c\\uc774\\ubd09\\ubc15\\ub4504: RUMOR","genre":"\\ub4dc\\ub77c\\ub9c8|\\ucf54\\ubbf8\\ub514|\\uacf5\\ud3ec","year":2015,"date":7.11,"rating":8.33,"vote_count":12,"plot":"[OPEN] \\uac10\\ub3c5 : \\ubc94, \\uc7a5\\ub974 : \\ub4dc\\ub77c\\ub9c8 \\uce5c\\uad6c\\uc640 \\ud568\\uaed8 \\uacfc\\uc81c\\ub97c \\ud558\\uace0 \\uc788\\ub294 \\ud55c\\ub3c4\\uc758 \\uc790\\ucde8\\ubc29\\uc73c\\ub85c \\uc220 \\ucde8\\ud55c \\uc560\\uc778\\uc774 \\uac11\\uc791\\uc2a4\\ub808 \\ucc3e\\uc544\\uc628\\ub2e4. \\uadf8\\ub807\\uac8c \\uc2dc\\uc791\\ub418\\ub294 \\ud55c\\ub3c4\\u00b7\\uce5c\\uad6c\\u00b7\\uc560\\uc778\\uc758 3\\uac01 \\uc904\\ub2e4\\ub9ac\\uae30. [FMSM] \\uac10\\ub3c5 : \\uace0\\uc218\\ubbf8, \\uc7a5\\ub974: \\ub4dc\\ub77c\\ub9c8 \\"\\uc5b4\\ub290 \\uac8c\\uc77

In [246]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

{'_shards': {'total': 15, 'successful': 15, 'failed': 0}}

In [ ]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d " % res["hits"]["total"]["value"])
count = aos_client.count(index=index_name)
print(count)

## Neural Search


In [437]:
# query_text = "우주에서 펼쳐지는 전쟁 이야기"
# query_text = "동물이 나오는 만화"
# query_text = "영웅들이 힘을 합쳐 빌런을 물리치고 세상을 구한다"
query_text = "큰 자연 재해가 발생하고 인류는 위기를 맞는다"

In [438]:
semantic_query = {
    "size": 30,
    "_source": {"excludes": ["title_vector, plot_vector, genre_vector"]},
    "query": {
        "bool": {
            # "must": [
            "should": [
                {
                    "neural": {
                        "title_vector": {
                            "query_text": query_text,
                            "model_id": model_id,
                            "k": 30,
                        }
                    }
                },
                {
                    "neural": {
                        "plot_vector": {
                            "query_text": query_text,
                            "model_id": model_id,
                            "k": 30,
                        }
                    }
                },
                {
                    "neural": {
                        "genre_vector": {
                            "query_text": query_text,
                            "model_id": model_id,
                            "k": 30,
                        }
                    }
                },
            ]
        }
    },
}

In [439]:
res = aos_client.search(index=index_name, body=semantic_query)
query_result = []
for hit in res["hits"]["hits"]:
    row = [
        hit["_id"],
        hit["_score"],
        hit["_source"]["title"],
        hit["_source"]["plot"],
        hit["_source"]["genre"],
        hit["_source"]["rating"],
    ]
    query_result.append(row)

query_result_df = pd.DataFrame(
    data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
)
display(query_result_df)

,_id,_score,title,plot,genre,rating
0,xT1Rjo4B1FbzESchYpha,0.010095,콜로니: 인류 최후의 날,우주에서 화물을 나르고 지구로 돌아가던 우주선 오셀롯은 불의의 사고로 소행성에 불시...,모험|SF|스릴러,1.24
1,nj1Qjo4B1FbzEScht5YT,0.009939,세상의 종말을 예언하는 소년: 프로디지,"2 24년 8월 15일, 전 세계인이 동시에 의식을 잃는 사건이 발생한다. 일종의 ...",드라마|스릴러,1.74
2,DT1Tjo4B1FbzESch3KBi,0.009832,콜로니:지구 최후의 날,"2144년, 갑작스러운 지구 온난화 이후 빙하기가 찾아왔다. 가까스로 살아남은 생존...",액션|SF|스릴러,6.30
3,1lZTjo4B5vU2gwgc8MJ7,0.009655,헬벤더스: 최후의 성전,"인류의 종말을 목전에 둔 순간, 잔혹한 악마들에게 맞서기 위해 그들이 나섰다! 악마...",공포|스릴러,5.54
4,1z1Tjo4B1FbzESchy5-C,0.009586,인투 더 스톰,갑작스런 기상 이변으로 발생한 수퍼 토네이도가 오클라호마의 실버톤을 덮쳐 쑥대밭으로...,액션|스릴러,8.23
5,zj1Ujo4B1FbzESchE6Bx,0.009466,디 엔드: 인류 최후의 날,"2 년 지기 친구들인 펠릭스, 사라, 휴고, 마리벨, 라파, 세르히오는 과거 우발적...",스릴러|모험,3.92
6,az1Ujo4B1FbzESchSqE3,0.009463,신밧드의 7대 모험,"세계 최대의 유조선 ‘아카바’호가 해적들에 납치당하자, 선박회사 ‘카마란’의 회장 ...",액션|판타지|모험,2.20
7,aFZUjo4B5vU2gwgcIMPM,0.009400,"마진 콜: 24시간, 조작된 진실",갑작스런 인원 감축으로 퇴직 통보를 받는 리스크 관리 팀장 에릭은 자신의 부하직원 ...,스릴러|드라마,7.76
8,6j1Tjo4B1FbzESch0Z8h,0.009282,존은 끝에 가서 죽는다,감히 비교 불가능의 찌질한 루저인 존과 데이빗은 속칭 ‘간장’으로 불리는 정체불명의...,판타지|공포|코미디,6.67
9,5j1Tjo4B1FbzESch0Z8h,0.009208,조난자들,여행이 스릴러가 된다! 홀로 깊은 산속 주인 없는 펜션을 찾아온 허세 여행자 ‘상진...,미스터리|스릴러,7.24


In [440]:
## Lexical (keyword) Search한 결과와 비교

In [441]:
lexical_query = {
    "size": 30,
    "query": {
        "multi_match": {
            "query": query_text,
            "fields": ["title", "plot", "genre"],
        }
    },
}

In [442]:
res = aos_client.search(index=index_name, body=lexical_query)
query_result = []
for hit in res["hits"]["hits"]:
    row = [
        hit["_id"],
        hit["_score"],
        hit["_source"]["title"],
        hit["_source"]["plot"],
        hit["_source"]["genre"],
        hit["_source"]["rating"],
    ]
    query_result.append(row)

query_result_df = pd.DataFrame(
    data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
)
display(query_result_df)

,_id,_score,title,plot,genre,rating
0,nz1Tjo4B1FbzESchu59v,18.694748,엣지 오브 투모로우,"가까운 미래, 미믹이라 불리는 외계 종족의 침략으로 인류는 멸망 위기를 맞는다. 빌...",액션|SF,8.95
1,cT1Qjo4B1FbzESchqZZI,16.245892,미용실 : 특별한 서비스 3,"동네의 한적한 미용실, 여성 디자이너 세정마저 일을 그만 두자 가게는 큰 위기를 맞...",멜로/로맨스,10.00
2,vj1Sjo4B1FbzESchD5qS,15.608034,현상금 사냥꾼,얼어붙은 불모의 땅 서부에서 살아가던 와이엇(제임스 랜슨)과 사무엘(조쉬 펙) 형제...,서부|액션|모험,4.00
3,S1ZWjo4B5vU2gwgcBslp,14.668031,북극의 눈물,MBC 창사 47주년 특별기획 '북극의 눈물'은 '세계 극지의 해'를 맞아 벼랑 끝...,다큐멘터리,8.20
4,IVZSjo4B5vU2gwgcqb_g,13.097054,감옥풍운,평생을 바른 생활 사나이로 살아오던 '노가요'(양가휘)는 아버지를 때린 이들과 싸우...,드라마|액션,7.42
5,KD1Ujo4B1FbzESchhqI3,12.455068,헬,"때는 2 16년. 지금으로부터 4년 후, 모든 생명의 근원이었던 태양의 흑점 폭발로...",SF|공포|스릴러,6.45
6,MT1Wjo4B1FbzESch6qmK,12.117457,샤크,"무소불위, 절대권력에 막강 카리스마를 가진 상어 대부 돈 리노. 하지만 그에게도 말...",애니메이션|가족|코미디|모험,7.04
7,41ZVjo4B5vU2gwgc48j2,12.016863,강적들,"수호는 대통령인 아버지(강민국)의 생일에도 어김없이 말썽을 일으키고, 보다 못한 영...",드라마|액션,8.00
8,Pj1Tjo4B1FbzESchPZ6i,11.316657,화산고래,"2 7 년, 인류는 대지진과 화산폭발로 위기를 맞게 되었다. 자연재해로 인해 무정부...",애니메이션,6.63
9,AD1Tjo4B1FbzESchhZ9m,10.261786,미 앤 유,"이 시대의 거장이 만든 아주 특별한 음악 성장 영화! “사랑하는 나의 동생, 널 위...",드라마|가족,7.93


## Hybrid Search를 해보자


### Search Pipeline 생성


In [443]:
search_pipeline_payload = {
    "description": "Post processor for hybrid search",
    "phase_results_processors": [
        {
            "normalization-processor": {
                "normalization": {"technique": "min_max"},
                "combination": {
                    "technique": "arithmetic_mean",
                    "parameters": {"weights": [0.3, 0.7]},
                },
            }
        }
    ],
}

search_pipeline_id = "search_pipeline"

In [447]:
# GET /my-nlp-index/_search?search_pipeline=nlp-search-pipeline
hybrid_query = {
    "size": 30,
    "_source": {"exclude": ["passage_embedding"]},
    "query": {
        "hybrid": {
            "queries": [
                {"match": {"plot": {"query": query_text}}},
                {
                    "neural": {
                        "plot_vector": {
                            "query_text": query_text,
                            "model_id": model_id,
                            "k": 30,
                        }
                    }
                },
            ]
        }
    },
    "search_pipeline": {
        "description": "Post processor for hybrid search",
        "phase_results_processors": [
            {
                "normalization-processor": {
                    "normalization": {"technique": "min_max"},
                    "combination": {
                        "technique": "arithmetic_mean",
                        "parameters": {"weights": [0.5, 0.5]},
                    },
                }
            }
        ],
    },
}

In [448]:
res = aos_client.search(index=index_name, body=hybrid_query)
query_result = []
for hit in res["hits"]["hits"]:
    row = [
        hit["_id"],
        hit["_score"],
        hit["_source"]["title"],
        hit["_source"]["plot"],
        hit["_source"]["genre"],
        hit["_source"]["rating"],
    ]
    query_result.append(row)

query_result_df = pd.DataFrame(
    data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
)
display(query_result_df)

,_id,_score,title,plot,genre,rating
0,KD1Ujo4B1FbzESchhqI3,0.632655,헬,"때는 2 16년. 지금으로부터 4년 후, 모든 생명의 근원이었던 태양의 흑점 폭발로...",SF|공포|스릴러,6.45
1,Yj1Wjo4B1FbzESchoKhj,0.568196,불편한 진실,"기상이변으로 인한 심각한 환경 위기! 킬리만자로, 몬타나 주 빙하국립공원, 콜롬비아...",다큐멘터리,8.79
2,Aj1Qjo4B1FbzESch2pfB,0.537729,유랑지구,"가까운 미래, 태양계 소멸 위기를 맞은 지구는 영하 7 도의 이상 기후와 함께 목성...",SF,5.89
3,kj1Vjo4B1FbzESchpqVk,0.522603,2012,"고대 마야 문명에서부터 끊임없이 회자되어 온 인류 멸망. 2 12년, 저명한 과학자...",액션|드라마|스릴러|SF|모험,7.61
4,nz1Tjo4B1FbzESchu59v,0.500000,엣지 오브 투모로우,"가까운 미래, 미믹이라 불리는 외계 종족의 침략으로 인류는 멸망 위기를 맞는다. 빌...",액션|SF,8.95
5,qj1Ujo4B1FbzESchXaFM,0.500000,월드워Z,"전 세계 이상 기류… 거대한 습격이 시작된다! 의문의 항공기 습격, 국가별 입국 전...",드라마|스릴러|SF|액션|모험,8.22
6,XFZYjo4B5vU2gwgcKM_z,0.447231,타이탄 AE,멀고도 광활한 은하계로의 여행은 이제 인류에게 일상적이고 자유로운 하나의 생활이 되...,애니메이션|모험|전쟁|SF|가족,7.92
7,dD1Rjo4B1FbzESch9Zpj,0.429835,지오스톰,"가까운 미래, 기후변화로 인해 지구에 갖가지 자연재해가 속출한다. 세계 정부 연합은...",액션|SF|스릴러,7.44
8,cT1Qjo4B1FbzESchqZZI,0.428060,미용실 : 특별한 서비스 3,"동네의 한적한 미용실, 여성 디자이너 세정마저 일을 그만 두자 가게는 큰 위기를 맞...",멜로/로맨스,10.00
9,XVZRjo4B5vU2gwgca7uv,0.415472,퍼스트맨,"이제껏 누구도 경험하지 못한 세계에 도전한 우주비행사 닐(라이언 고슬링)은, 거대한...",SF|드라마,6.96
